In [2]:
import os, transformers
import csv
import spacy
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertTokenizerFast
import opencc
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import random

converter = opencc.OpenCC('s2tw.json')

# Download data
# data_utils.download_data("./")

# Load model with CPU
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

2022-12-26 22:59:18.452053: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-26 22:59:18.694635: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-26 22:59:18.714310: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-26 22:59:18.714330: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [3]:
# cn_version
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# zh-tw version
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['cls.predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [105]:
# generate label

def generate_label(sentence,tag_pos,p):

    sentence_list = [sentence]
    word_sentence_list = ws(sentence_list)
    pos_sentence_list = pos(word_sentence_list)

    label_list = []

    for i in range(len(word_sentence_list)):
        for r in range(len(word_sentence_list[i])):
            word_ = word_sentence_list[i][r]
            pos_ = pos_sentence_list[i][r]

            label_list.append([word_,pos_])

    # get tag's next word's position
    def get_key(val):
        result = []
        position = 0
        del_list = []
        for i in range(len(label_list)):
            key, value = label_list[i]
            position += len(key)
            
            if val == value:

                # judge del or save
                # p = 0.9
                p_ = random.randint(1,100)/100
                if p_ <= p:
                    # del
                    position -= len(key)
                    result.append([0,position])
                    del_list.append(i)
                    # print(0,position, sentence_list[0][position])
                else:
                    # save
                    result.append([1,position])
                    # print(1,position, sentence_list[0][position])
        
        # print("".join([x[0] for x in label_list]))
        del_list.sort(reverse=True)
        for _ in del_list:
            del label_list[_]
        sent = "".join([x[0] for x in label_list])
                    
        return result, sent

    label_tag, sent = get_key(tag_pos)
    encoded_str = tokenizer(sent, padding=True, truncation=True) 
    tokens = tokenizer.convert_ids_to_tokens(encoded_str.input_ids)

    lengh = len(tokens[1:-1])
    label = ''

    while len(label)<lengh:
        label+='0'


    for i in [r[1] for r in label_tag]:
        label = list(label)
        label[i] = '1'
        label = ''.join(label)

    # print(sentence_list[0])
    # print(sent)

    return sent, label

In [106]:
generate_label('傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。','D',0.5)

('傅達仁今執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他懂自己哪裡得罪到電視台。',
 '000010000001010000000000000000010000000000')